In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%%capture
!pip install av
!pip install transformers

In [ ]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import numpy as np
import gc
from tqdm import tqdm
import av
from transformers import VivitImageProcessor, VivitModel

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/LVD.csv')

In [ ]:
videos = df['name'].apply( lambda x :'video'+ x + '.mp4')

In [ ]:
np.random.seed(0)
def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])
image_processor = VivitImageProcessor.from_pretrained("google/vivit-b-16x2-kinetics400")
model = VivitModel.from_pretrained("google/vivit-b-16x2-kinetics400")

Some weights of VivitModel were not initialized from the model checkpoint at google/vivit-b-16x2-kinetics400 and are newly initialized: ['vivit.pooler.dense.bias', 'vivit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
index_frames_matrix = np.load('/content/gdrive/MyDrive/Projet_Multimedia/download/KeyImages/FinalMatrixFramesIndices.npy')

In [ ]:
max = 2210

In [ ]:
interval = 2080+13+13+13+13+13+13+13+13+13
vid_embeddings = np.zeros((13, 3137, 768))

In [ ]:
interval

2197

In [ ]:
del df

In [ ]:
vid_folder_path = "/content/gdrive/MyDrive/Projet_Multimedia/videos/Video_with_label/"
for count, vid_name in tqdm(enumerate(videos)):
  if interval <= count  and count < interval+13 and count < max:
    indices = index_frames_matrix[count]
    if len(set(indices)) == 32:
      container = av.open('/content/gdrive/MyDrive/Projet_Multimedia/videos/Video_with_label/' + vid_name)
      video = read_video_pyav(container=container, indices=indices)
      container.close()
      inputs = image_processor(list(video), return_tensors="pt")


      del video
      del indices
      gc.collect() ## Clean the bin
      outputs = model(**inputs)
      vid_embeddings[count-interval] = outputs.last_hidden_state.detach().numpy()

      del outputs ## Ram problem -> put it in the bin
      gc.collect() ## Clean the bin
      print(count)
  #if count % 200 == mod:
  #  np.save('/content/gdrive/MyDrive/Projet_Multimedia/download/Video_Embeddings/VideoEmbeddings' + str(mod) + '.npy',vid_embeddings)
np.save('/content/gdrive/MyDrive/Projet_Multimedia/download/Video_Embeddings/VideoEmbeddings' + str(interval) + '.npy',vid_embeddings)

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  return torch.tensor(value)
2198it [00:59, 37.14it/s]

2197


2199it [01:32, 20.61it/s]

2198


2200it [02:04, 12.74it/s]

...

2207


2209it [06:52,  2.28s/it]

2208


2280it [07:24,  5.13it/s]

2209
